In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "5.Coliadinae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Colias_behrii
Exon_1 Match
[['Zerene_cesonia', ['6804607', '6804703']], ['Pieris_brassicae', ['6804607', '6804703']], ['Maniola_jurtina', ['6804607', '6804703']], ['Colias_croceus', ['6804607', '6804703']]]
Exon_2 Match
[['Zerene_cesonia', ['6805578', '6806125']], ['Pieris_brassicae', ['6805578', '6806125']], ['Maniola_jurtina', ['6805578', '6806125']], ['Colias_croceus', ['6805578', '6806125']]]
Exon_3 Match
[['Zerene_cesonia', ['6806635', '6806788']], ['Pieris_brassicae', ['6806635', '6806788']], ['Maniola_jurtina', ['6806635', '6806788']], ['Colias_croceus', ['6806635', '6806788']]]
Exon_4 mismatch
[['Zerene_cesonia', ['6806922', '6807151']], ['Pieris_brassicae', ['6806922', '6807151']], ['Maniola_jurtina', ['6806922', '6807169']], ['Colias_croceus', ['6806922', '6807151']]]
Proceed?y
Zerene_cesonia Exon_4 18_frame2 0.205 
SDDSSPMLTSDPTSDSSDTGGSGKMNDESNSPSNSWDTSRNISEQNFPSDRREDKSSSHMLDSGQENSYDDMKQSN
Pieris_brassicae Exon_4 18_frame2 0.963 
SDDSSPMLTSDPTSDSSDTGGSGKMNDESNSPSNSWDTSRNISE

Choose Species 1 - 41
Exon_16 Match
[['Zerene_cesonia', ['6813178', '6813361']], ['Pieris_brassicae', ['6813178', '6813361']], ['Maniola_jurtina', ['6813178', '6813361']], ['Colias_croceus', ['6813178', '6813361']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Colias_behrii,JARWMC010000003.1,6804607,6804703,0,N,Colias_croceus_XM_045654792.1_Frame_0_rightoh_1_query_Exon_1,1,32,32
Colias_behrii,JARWMC010000003.1,6805578,6806125,0,N,Zerene_cesonia_XM_038355360.1_Frame_2_rightoh_0_query_Exon_2,1,182,182,Y,N
Colias_behrii,JARWMC010000003.1,6806635,6806788,0,N,Zerene_cesonia_XM_038355360.1_Frame_0_rightoh_1_query_Exon_3,1,51,51,Y,N
Colias_behrii,JARWMC010000003.1,6806922,6807151,0,N,Zerene_cesonia_XM_038355360.1_Frame_2_rightoh_0_query_Exon_4,1,76,76,Y,N
Colias_behrii,JARWMC010000003.1,6807307,6807602,0,N,Zerene_cesonia_XM_038355360.1_Frame_0_rightoh_2_query_Exon_5,1,98,98,Y,N
Colias_behrii,JARWMC010000003.1,6807688,6808048,0,N,Zerene_cesonia_XM_03835

Proceed?y
Zerene_cesonia Exon_10 21_frame1 0.0 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKD
Pieris_brassicae Exon_10 21_frame1 0.526 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKD
Maniola_jurtina Exon_10 21_frame1 1.089 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNED
Colias_croceus Exon_10 21_frame1 0.091 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKD
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ ----DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSS
set21_frame1    NLMQDWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSS
                    ********************************************************

Zerene_cesonia_ RAIVNYDASVKD----
set21_frame1    RAIVNYDASVKDRYYI
                ************    

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica ----DWSDLIEYVINTFIKKPHSLQRTASFRLDTESFDDDKVRKDLIDDSKEFMLDVKDT
set21_frame1    NLMQDWSDLIEYVINTFVK

Proceed?y
Zerene_cesonia Exon_15 18_frame1 1.271 
TIIQTKGAD
Pieris_brassicae Exon_15 18_frame1 1.701 
TIIQTKGADQLNNSDAQEVTLSSEIKTCKIVSALRSPELIEECDIVTEMSLLIPVDGDSKREPSPFDMDQADSEYCN
Maniola_jurtina Exon_15 18_frame1 1.589 
TIIQTKGADQLNNSDAQEVTLSSEIKTCKIVSALRSPELIEECDIVTEMSLLIPVDGDSKREPSPFDMDQADSEYCN
Colias_croceus Exon_15 18_frame1 1.195 
TIIQTKGADQLNNSDAQEVTLSSEIK
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ -------------------------------------------TIIQSEIQD--------
set18_frame1    LARCRSEISRIQIKFGASKVSIRRATGIIHRVVMLFNLYFYFRTIIQTKGADQLNNSDAQ
                                                           ****::  *        

Zerene_cesonia_ ---MSKLVGKEPTANTLRILEINKEECDVAAESPLLIPIDNGVKGEPSNFNIGHAESEYC
set18_frame1    EVTLSSEIKTCKIVSALRSPEL-IEECDIVTEMSLLIPVDGDSKREPSPFDMDQADSEYC
                   :*. : .   ..:**  *:  ****:.:* .****:*.. * *** *::.:*:****

Zerene_cesonia_ NN
set18_frame1    NR
                *.

CLUSTAL format alignment by MAFFT L-INS-

Choose Species 1 - 42
Exon_6 Match
[['Zerene_cesonia', ['346974', '347367']], ['Pieris_brassicae', ['346974', '347367']], ['Maniola_jurtina', ['346974', '347367']], ['Colias_croceus', ['346974', '347367']]]
Exon_7 mismatch
[['Zerene_cesonia', ['347898', '348233']], ['Pieris_brassicae', ['347898', '348233']], ['Maniola_jurtina', ['347898', '348233']], ['Colias_croceus', ['347050', '347463']]]
Proceed?y
Zerene_cesonia Exon_7 34_frame1 1.781 
YLVSENRILYCRWTDDELSAFNWIYMQSSTSTDIVAEIVRLFKEDGIIKTRDAVIRQLHKQSYIDKNEFDKLIKEESGKSSKTSETKEVKDDDIGKLCEYMTKDGKSKFLDW
Pieris_brassicae Exon_7 34_frame1 1.722 
YLVSENRILYCRWTDDELSAFNWIYMQSSTSTDIVAEIVRLFKEDGIIKTRDAVIRQLHKQSYIDKNEFDKLIKEESGKSSKTSETKEVKDDDIGKLCEYMTKDGKSKFLDW
Maniola_jurtina Exon_7 34_frame1 1.721 
YLVSENRILYCRWTDDELSAFNWIYMQSSTSTDIVAEIVRLFKEDGIIKTRDAVIRQLHKQSYIDKNEFDKLIKEESGKSSKTSETKEVKDDDIGKLCEYMTKDGKSKFLDW
Colias_croceus Exon_7 13_frame2 1.783 
DFLLKGYNSLVRTLHTQILTNLQLEIDTSHFFWLVTYFLKFATQIELDLEHVSSVLSFDIVLYLTAEGVNLCEQFELAIKLDGNDLKPSIRRLHXXX

...

Proceed?y
Zerene_cesonia Exon_10 17_frame2 0.0 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKD
Pieris_brassicae Exon_10 17_frame2 0.526 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKD
Maniola_jurtina Exon_10 17_frame2 1.089 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKDRYDRMIINLTYTC
Colias_croceus Exon_10 17_frame2 0.091 
DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIVNYDASVKD
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIV
set17_frame2    DWSDLIEYVINTFVKKPHTLQRTASFRLDTESFDDDKILKELIENEEKPNEDVSSSRAIV
                ************************************************************

Zerene_cesonia_ NYDASVKD---------------------
set17_frame2    NYDASVKDRYDRMIINLTYTCMFILQIMR
                ********                     

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica DWSDLIEYVINTFIKKPHSLQRTASFRL

Proceed?y
Zerene_cesonia Exon_4 41_frame2 1.356 
SSDSPMLSSGPTSDCSDNEGSAKMKGQSDSPTNSWDSVIRSTNNDTEKLYKSNNTMKPKPFNKSVSAMDISLSFDENQSD
Pieris_brassicae Exon_4 41_frame2 1.403 
SSDSPMLSSGPTSDCSDNEGSAKMKGQSDSPTNSWDSVIRSTNNDTEKLYKSNNTMKPKPFNKSVSAMDISLSFDENQSD
Maniola_jurtina Exon_4 41_frame2 1.447 
SSDSPMLSSGPTSDCSDNEGSAKMKGQSDSPTNSWDSVIRSTNNDTEKLYKSNNTMKPKPFNKSVSAMDISLSFDENQSDVSIYSI
Colias_croceus Exon_4 41_frame2 1.353 
SSDSPMLSSGPTSDCSDNEGSAKMKGQSDSPTNSWDSVIRSTNNDTEKLYKSNNTMKPKPFNKSVSAMDISLSFDENQSDVSIYSI
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ ------------SDDSSPMLTSDPTSDSSDTGGSGKMNDESNSPNNGWD----TSRNISE
set41_frame2    YPLLINILCVFVGSSDSPMLSSGPTSDCSDNEGSAKMKGQSDSPTNSWDSVIRSTNNDTE
                            ....****:*.****.**. **.**:.:*:**.*.**    ::.* :*

Zerene_cesonia_ QNFPSDR-MEEKTSNHMLDSGQEN-SYDEMKKT---------------------------
set41_frame2    KLYKSNNTMKPKPFNKSVSAMDISLSFDENQSDVSIYSIVCFYLISTPFSKVHFRCCMFL
                : : *:. *: *. *: :.: 

Choose Species 1 - 42
Exon_16 Match
[['Zerene_cesonia', ['11730487', '11730553']], ['Pieris_brassicae', ['11730487', '11730553']], ['Maniola_jurtina', ['11730487', '11730553']], ['Colias_croceus', ['11730487', '11730553']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Leptidea_juvernica,CATIXQ010000011.1,11753168,11753267,1,N,Colias_croceus_XM_045654792.1_Frame_0_rightoh_1_query_Exon_1,3,32,32
Leptidea_juvernica,CATIXQ010000011.1,11751906,11752462,1,N,Zerene_cesonia_XM_038355360.1_Frame_2_rightoh_0_query_Exon_2,1,185,185,Y,N
Leptidea_juvernica,CATIXQ010000011.1,11749556,11749709,1,N,Zerene_cesonia_XM_038355360.1_Frame_0_rightoh_1_query_Exon_3,1,51,51,Y,N
Leptidea_juvernica,CATIXQ010000011.1,11749230,11749471,1,N,Zerene_cesonia_XM_038355360.1_Frame_2_rightoh_0_query_Exon_4,1,80,80,Y,N
Leptidea_juvernica,CATIXQ010000011.1,11748163,11748446,1,N,Zerene_cesonia_XM_038355360.1_Frame_0_rightoh_2_query_Exon_5,1,94,94,Y,N
Leptidea_juvernica,CATIXQ0100000

Choose Species 1 - 43
Exon_16 Match
[['Zerene_cesonia', ['13808440', '13808506']], ['Pieris_brassicae', ['13808440', '13808506']], ['Maniola_jurtina', ['13808440', '13808506']], ['Colias_croceus', ['13808440', '13808506']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Leptidea_reali,CATIXK010000015.1,13842846,13842945,1,N,Colias_croceus_XM_045654792.1_Frame_0_rightoh_1_query_Exon_1,3,32,32
Leptidea_reali,CATIXK010000015.1,13841572,13842140,1,N,Zerene_cesonia_XM_038355360.1_Frame_2_rightoh_0_query_Exon_2,1,189,189,Y,N
Leptidea_reali,CATIXK010000015.1,13839227,13839380,1,N,Zerene_cesonia_XM_038355360.1_Frame_0_rightoh_1_query_Exon_3,1,51,51,Y,N
Leptidea_reali,CATIXK010000015.1,13838901,13839142,1,N,Zerene_cesonia_XM_038355360.1_Frame_2_rightoh_0_query_Exon_4,1,80,80,Y,N
Leptidea_reali,CATIXK010000015.1,13837753,13838036,1,N,Zerene_cesonia_XM_038355360.1_Frame_0_rightoh_2_query_Exon_5,1,94,94,Y,N
Leptidea_reali,CATIXK010000015.1,13836846,13837206,1

Maniola_jurtina Exon_14 65_frame0 1.088 
PGSSCDSPSNERSSSSWLQVVQKAQELNLPLEIE
Colias_croceus Exon_14 65_frame0 0.949 
PGSSCDSPSNERSSSSWLQVVQKAQELNLPLEIE
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ -----PGSSFNIACDERSVPSWLDVVQKSQEFKLTLELE------------
set65_frame0    YLYFRPGSSCDSPSNERSSSSWLQVVQKAQELNLPLEIERLVTSKRTIGLQ
                     **** : ..:*** .***:****:**::*.**:*            

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica ---------------ETTGAGWLQVVQKSHEIKFTMDRE------------
set65_frame0    YLYFRPGSSCDSPSNERSSSSWLQVVQKAQELNLPLEIERLVTSKRTIGLQ
                               * :.:.*******::*:::.:: *            

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina -----PGSSFTTTDDNVTKPSWLEVVQKSQEYKITLDL------------G
set65_frame0    YLYFRPGSSCDSPSNERSSSSWLQVVQKAQELNLPLEIERLVTSKRTIGLQ
                     ****  :..:: :..***:****:** ::.*::             

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Col

In [ ]:
1